In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle

from xgboost import XGBRegressor
from IPython.display import clear_output

import itertools
import tensorflow
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Model
import time
# from sklearn.gaussian_process import GaussianProcessRegressor


In [37]:
### read all the simulation data with different properties
df1 = pd.read_csv("Data/FE_constAng3_29param.csv")

df2 = pd.read_csv("Data/FE_Var_SizeAngle3_29param.csv")

df3 = pd.read_csv("Data/FE_Var_SizeConstAng3_29param.csv")

df4 = pd.read_csv("Data/FE_varAng3_29param.csv")

df9 = pd.read_csv("Data/FE_constAng3_repeat_29param.csv")

df = pd.concat([df1, df2, df3, df4, df9], axis = 0, ignore_index = True)
df.shape
# df = shuffle(df)
# df5 = pd.read_csv("Data/FE_constAng5_29param.csv")

# df6 = pd.read_csv("Data/FE_varAng5_29param.csv")

# df7 = pd.read_csv("Data/FE_constAng7_29param.csv")

# df8 = pd.read_csv("Data/FE_varAng7_29param.csv")



# df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8 , df9
#                        ], axis = 0, ignore_index = True)

# df = shuffle(df)
######### or filtered data
# df = pd.read_csv("Data/FE_filtered_29param.csv")

(77, 29)

In [11]:
df.loc[(df.FricDissipRate > 4)].index[0]

51

In [12]:
df.loc[(df.Safety_factor < 0.015)].index[0]

32

In [13]:
df.loc[(df.Tot_contactEngy > 50)].index[0]

94

In [14]:
df_filtered = df.drop([df.loc[(df.Safety_factor < 0.015)].index[0],
                      df.loc[(df.FricDissipRate > 4)].index[0],
                      df.loc[(df.Tot_contactEngy > 50)].index[0]
                      ], axis = 0)
df = df_filtered.reset_index(drop=True)

In [15]:
# ###------- normalizing data 
normalizer = MinMaxScaler()
df_norm = normalizer.fit_transform(df)

df = pd.DataFrame(df_norm, columns=df.columns )
print(df.keys())

Index(['Number_pieces', 'Length_ratio', 'angle1_9p', 'angle2_9', 'angle3_9p',
       'angle4_9p', 'angle1_25p', 'angle2_25p', 'angle3_25p', 'angle4_25p',
       'angle5_25p', 'angle6_25p', 'angle1_49p', 'angle2_49p', 'angle3_49p',
       'angle4_49p', 'angle5_49p', 'angle6_49p', 'angle7_49p', 'angle8_49p',
       'Safety_factor', 'Oop_deform', 'Tot_contactEngy', 'Elast_strainEngy',
       'Edge_temp', 'Avr_frictForce', 'HeatRate', 'IntEngy', 'FricDissipRate'],
      dtype='object')


In [16]:
# df_clean3 = df.drop(df.iloc[:,6:20], axis = 'columns')
X = df.iloc[:, 0:6]
Y1 = df.iloc[:, 20:21]      ## Safety_factor
Y2 = df.iloc[:, 21:22]       ## Oop_deform
Y3 = df.iloc[:, 22:23]     ## Tot_contactEngy
Y4 = df.iloc[:, 23:24]     ## Elast_strainEngy
Y5 = df.iloc[:, 24:25]     ## Edge_temp
Y6 = df.iloc[:, 25:26]     ## Avr_frictForce
Y7 = df.iloc[:, 26:27]     ## HeatRate
Y8 = df.iloc[:, 27:28]     ## IntEngy
Y9 = df.iloc[:, 28:29]     ## FricDissipRate


In [17]:
def defineTree(Xtrain, Ytrain):
    XGB_Dpt = XGBRegressor(max_depth=3,                 # Depth of each tree
                                learning_rate=0.1,            # How much to shrink error in each subsequent training. Trade-off with no. estimators.
                                n_estimators=100,             # How many trees to use, the more the better, but decrease learning rate if many used.
                                verbosity=0,                  # If to show more errors or not.
                                objective='reg:squarederror',  # Type of target variable. for classifieer use 'binary:logistic'
                                n_jobs=4,                     # Parallel jobs to run. Set your processor number.
                                gamma=0.001,                  # Minimum loss reduction required to make a further partition on a leaf node of the tree. (Controls growth!)
                                subsample=1,              # Subsample ratio. Can set lower than 1. If we want perfect boosting
                                colsample_bytree=1,           # Subsample ratio of columns when constructing each tree.
                                colsample_bylevel=1,          # Subsample ratio of columns when constructing each level. 0.33 is similar to random forest. sqrt(no.var)
                                colsample_bynode=1,           # Subsample ratio of columns when constructing each split.
                                scale_pos_weight=1,           # Balancing of positive and negative weights.
                                base_score=0.5,               # Global bias. Set to average of the target rate.
                                random_state=20210614,        # Seed
                                missing=1                  # How are nulls encoded?
                                )

    param_grid = dict({'n_estimators': [20, 50, 100],
                        'max_depth': [3, 5, 7],
                        'learning_rate': [0.01, 0.1, 0.5],
                        'subsample': [0.632, 1.],
                        'base_score' : [0.1, 0.5, 1]
                      })


    # Define grid search object.
    GridXGB = GridSearchCV(XGB_Dpt,        # Original XGB.
                           param_grid,          # Parameter grid
                           cv = 3,              # Number of cross validation folds.
                           n_jobs = 4,          # Parallel jobs. -1 is "all you have"
                           refit = False,       # If refit at the end with the best. We'll do it manually
                           verbose = 1           # If to show what it is doing
                          )

    # Train grid search.
    GridXGB.fit(Xtrain, Ytrain)
    return GridXGB.best_params_

In [27]:
out1 = defineTree(X, Y1)
print('Safety_factor\n',out1)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Safety_factor
 {'base_score': 0.1, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.632}


In [28]:
out2 = defineTree(X, Y2)
print('Oop_deform\n',out2)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Oop_deform
 {'base_score': 1, 'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 20, 'subsample': 0.632}


In [29]:
out3 = defineTree(X, Y3)
print('Tot_contactEngy\n',out3)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Tot_contactEngy
 {'base_score': 1, 'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 20, 'subsample': 0.632}


In [30]:
out4 = defineTree(X, Y4)
print('Elast_strainEngy\n',out4)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Elast_strainEngy
 {'base_score': 0.5, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 50, 'subsample': 0.632}


In [32]:
out5 = defineTree(X, Y5)
print('Edge_temp\n',out5)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Edge_temp
 {'base_score': 0.1, 'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.632}


In [33]:
out6 = defineTree(X, Y6)
print('Avr_frictForce\n',out6)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Avr_frictForce
 {'base_score': 0.5, 'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}


In [34]:
out7 = defineTree(X, Y7)
print('HeatRate\n',out7)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
HeatRate
 {'base_score': 0.5, 'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}


In [35]:
out8 = defineTree(X, Y8)
print('IntEngy\n',out8)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
IntEngy
 {'base_score': 1, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.632}


In [36]:
out9 = defineTree(X, Y9)
print('FricDissipRate\n',out9)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
FricDissipRate
 {'base_score': 0.5, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 1.0}


In [30]:
def XGB_BestParam(Xtrain, Ytrain, Xtest, Ytest, maxDepth, learningRate, nEstimators, subSamples, baseScore):
    XGB = XGBRegressor(max_depth = maxDepth,                 # Depth of each tree
                        learning_rate = learningRate,            # How much to shrink error in each subsequent training. Trade-off with no. estimators.
                        n_estimators = nEstimators,             # How many trees to use, the more the better, but decrease learning rate if many used.
                        verbosity=1,                  # If to show more errors or not.
                        objective='reg:squarederror',  # Type of target variable. for classifieer use 'binary:logistic'
                        n_jobs=4,                     # Parallel jobs to run. Set your processor number.
                        gamma=0.001,                  # Minimum loss reduction required to make a further partition on a leaf node of the tree. (Controls growth!)
                        subsample = subSamples,              # Subsample ratio. Can set lower than 1. If we want perfect boosting
                        colsample_bytree=1,           # Subsample ratio of columns when constructing each tree.
                        colsample_bylevel=1,          # Subsample ratio of columns when constructing each level. 0.33 is similar to random forest. sqrt(no.var)
                        colsample_bynode=1,           # Subsample ratio of columns when constructing each split.
                        scale_pos_weight=0.5,           # Balancing of positive and negative weights.
                        base_score = baseScore,               # Global bias. Set to average of the target rate.
                        random_state=20210614,        # Seed
                        missing=1                  # How are nulls encoded?
                        )
    XGB.fit(Xtrain, Ytrain)
    predict = XGB.predict(Xtest)
#     loss = mean_squared_error(Ytest, predict)
    R2 = r2_score(Ytest, predict)
    return R2

In [39]:
# out = defineTree(Xtrain, Ytrain)
# print(out)

# baseScore = out['base_score']
# subSamples = out['subsample']
# nEstimators = out['n_estimators']
# learningRate = out['learning_rate']
# maxDepth = out['max_depth']

# loss, R2 = XGB_BestParam(Xtrain, Ytrain, Xtest, Ytest, maxDepth, learningRate, nEstimators, subSamples, baseScore)
# print(R2)

In [36]:
R2s = []
for i in range(5):
    Xtrain, Xtest,Ytrain, Ytest,  =\
            train_test_split(X, Y2 ,
            test_size = 0.02, random_state = i)
    out = defineTree(Xtrain, Ytrain)
    print('\n',out)

    baseScore = out['base_score']
    subSamples = out['subsample']
    nEstimators = out['n_estimators']
    learningRate = out['learning_rate']
    maxDepth = out['max_depth']

    R2 = XGB_BestParam(Xtrain, Ytrain, Xtest, Ytest, maxDepth, learningRate, nEstimators, subSamples, baseScore)
    R2s.append(R2)

Fitting 3 folds for each of 162 candidates, totalling 486 fits

 {'base_score': 0.1, 'learning_rate': 0.5, 'max_depth': 7, 'n_estimators': 20, 'subsample': 0.632}
Fitting 3 folds for each of 162 candidates, totalling 486 fits

 {'base_score': 0.5, 'learning_rate': 0.5, 'max_depth': 7, 'n_estimators': 20, 'subsample': 1.0}
Fitting 3 folds for each of 162 candidates, totalling 486 fits

 {'base_score': 1, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1.0}
Fitting 3 folds for each of 162 candidates, totalling 486 fits

 {'base_score': 0.1, 'learning_rate': 0.5, 'max_depth': 7, 'n_estimators': 20, 'subsample': 1.0}
Fitting 3 folds for each of 162 candidates, totalling 486 fits

 {'base_score': 1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 1.0}


In [35]:
R2s

[0.31403163527999967,
 0.23232620578271568,
 0.9423009562615317,
 0.3358492579473742,
 -28.740166405585533]